In [ ]:
import pandas as pd
import numpy as np
import h5py as h5
from glob import glob
import requests
import json
import time
from collections import defaultdict
import os
import pyarrow.feather as feather
from matplotlib import pyplot as plt
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()

In [ ]:
species = "human"
h5_version = 2.2
gsm4sig_version = 2

In [ ]:
f = h5.File(f"../{species}_gene_v{h5_version}.h5", "r")
genes = [x.decode('UTF-8') for x in f['meta/genes/symbol']]
f.close()

In [ ]:
gsm4sig = pd.read_csv(f"data/{species}_gsm4sig_v{gsm4sig_version}.csv", index_col=0)
gsm4sig["sig_num"] = gsm4sig.groupby("series_id").cumcount()+1

In [ ]:
len(gsm4sig["series_id"].unique())

In [ ]:
gsm4sig

In [ ]:
gse2title = defaultdict(lambda: "")
gse_trunc = gsm4sig["series_id"].str.split("-").map(lambda x: x[0]) #keeps only subseries if sub/superseries exists
api_key = os.getenv('API_KEY')              #use of api_key increases rate limit 3 -> 10 req/s
for i, gse in tqdm(enumerate(gse_trunc.unique())):                        #find title of the study via the ncbi e-utilities api                          #for use as part of signature name
    try:
        if ',' in gse:
            gse = gse.split(',')[0]
        url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
        params = {"db":"gds", 
                  "term": f"{gse}[ACCN]", 
                  "retmax":1, 
                  "retmode":"json",
                  "api_key":api_key}
        r = requests.get(url + "esearch.fcgi", params=params)
        params["id"] = r.json()["esearchresult"]["idlist"][0]
        r = requests.get(url + "esummary.fcgi", params=params)
        gse2title[gse] = r.json()['result'][params["id"]]["title"]
        del params["id"]
        time.sleep(1/6)
    except:
        print(i, gse)

In [ ]:
set(gse2title.keys()).symmetric_difference(set(gse_trunc.unique())) 
#studies in which the study name was not found (data privatized, etc)

In [ ]:
f = open(f"{species}_gse2title.json","w")
f.write(json.dumps(gse2title))
f.close()

In [ ]:
logFC = pd.DataFrame(index = genes)
pval = pd.DataFrame(index = genes)
adjpval = pd.DataFrame(index = genes)

glob_csv = glob(f"./results/{species}/*.csv")
for n, file in enumerate(glob_csv):                                                 #aggregate logFC, p-value, and adjusted
    if n % 200 == 0: print(n, "done")                                               #p-value from each individual signature
    if n % 500 == 0 and n != 0:                                                     #file into their own dataframe files for
        logFC.to_csv(f"./results/sub_{species}/{species}_logFC_chunk_{n}.csv")      #easy query later with appyter.
        pval.to_csv(f"./results/sub_{species}/{species}_pval_chunk_{n}.csv")        #files written at 500 signature increments
        adjpval.to_csv(f"./results/sub_{species}/{species}_adjpval_chunk_{n}.csv")  #to speed up aggregation
        
        logFC = pd.DataFrame(index = genes)
        pval = pd.DataFrame(index = genes)
        adjpval = pd.DataFrame(index = genes)
        
    idx, gse = file.split("\\")[-1].split("_")[:2]
    gse = gse.split("-")[0]
    colname = f"{gse2title[gse]} {gse}_{gsm4sig.at[int(idx), 'sig_num']}"
    file_df = pd.read_csv(file, index_col=0)
    
    logFC = (logFC
             .join(file_df["logFC"], how="left")
             .rename(columns={"logFC":colname})
             .fillna(0))

    pval = (pval
            .join(file_df["P.Value"], how="left")
            .rename(columns={"P.Value":colname})
            .fillna(1))
    
    adjpval = (adjpval
               .join(file_df["adj.P.Val"], how="left")
               .rename(columns={"adj.P.Val":colname})
               .fillna(1))    
    
logFC.to_csv(f"./results/sub_{species}/{species}_logFC_chunk_end.csv")
pval.to_csv(f"./results/sub_{species}/{species}_pval_chunk_end.csv")
adjpval.to_csv(f"./results/sub_{species}/{species}_adjpval_chunk_end.csv")

In [ ]:
data_type = [("logFC", "fc"), ("pval", "pval"), ("adjpval","adjpval")]
for data, title in data_type:                                                    #500-increment files for logFC, p-value
    glob_chunk = sorted(glob(f"results/sub_{species}/{species}_{data}_*.csv"),   #and adjusted p-value aggregated together
                        key=os.path.getmtime)                                    #for one file each
    pd.concat((pd.read_csv(file, index_col=0) for file in glob_chunk), axis=1).to_csv(f"all_{species}_{title}.csv")
    print("done with", data)

In [ ]:
species_list = ["human", "mouse"]
data_list = ["fc", "pval", "adjpval"]

for s in species_list:                                                           #csv files written as feather files to 
    for dat in data_list:                                                        #speed up reading speed during appyter query
        df = pd.read_csv(f"all_{s}_{dat}.csv", index_col=0)
        feather.write_feather(
            df.T.reset_index().rename(columns={'index': 'sig'}), 
            f"all_{s}_{dat}.feather"
        )
        print(f"all_{s}_{dat}.csv")

In [ ]:
species_list = ["human", "mouse"]
data_list = ["fc", "pval", "adjpval"]
num_splits = 350

for s in species_list:                                                         #files split up into 350 files of ~100 genes
    lookup_dict = {}                                                           #each in order to speed up reading during appyter
    for n, dat in enumerate(data_list):                                        #query and make lookup json file to identify
        big_df = pd.read_csv(f"all_{s}_{dat}.csv", index_col = 0)              #correct file to read given a queried gene
        dfs = np.array_split(big_df, num_splits)
        for i, df in enumerate(dfs):
            if n == 0: lookup_dict.update({idx:i for idx in df.index})
            df.T.to_csv(f"split_datafiles/{s}/{s}_{dat}/{s}_{dat}_{i}.csv")
        print(f'done with {s} {dat}')
    
    with open(f'{s}_lookup.json', 'w') as f:
        json.dump(lookup_dict, f)   
    print(f'done with {s}')

In [ ]:
print(f"Signature statistics for {species}:\n")                                #generate summary statistics for the given 
print(f"Total Signatures: {len(gsm4sig['extrap_score'])}")                     #human/mouse signatures
print(f"High quality signatures (score=0,1): \
{sum(gsm4sig['extrap_score'].value_counts()[[0,1]])/sum(gsm4sig['extrap_score'].value_counts())*100:.2f}%")
print(f"Breakdown by extrap_score:\n{gsm4sig['extrap_score'].value_counts()}")
plt.bar(gsm4sig["extrap_score"].value_counts().index, gsm4sig["extrap_score"].value_counts())
plt.show()